In [1]:
import pandas as pd
import sys
import itertools
import math
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from ast import literal_eval
from collections import Counter

sys.path.append('..')
from finder import *

In [2]:
fixes = pd.read_csv(PATH_VULNERABILITY_FIXES['fixes_labels_round_2'], index_col=False)

In [3]:
len(fixes.repository.unique())

994

In [4]:
pr_vuln = pd.read_csv(PATH_REPOSITORIES_DATA['pr_vulnerabilities'], index_col=False)

In [5]:
len(pr_vuln.repository.unique())

1004

In [6]:
missing_repos = [x for x in pr_vuln.repository.unique().tolist() if x not in fixes.repository.unique().tolist()]

In [7]:
missing_repos

['strapi/foodadvisor',
 'refugies-info/karfur',
 'imvetri/ui-editor',
 'sharetribe/flex-docs',
 'hull/hull-connectors',
 '18xx-maker/18xx-maker',
 'alleyinteractive/irving',
 'WeBankFinTech/wxa',
 'mistval/kotoba',
 'evt-project/evt-viewer']

In [9]:
len(pr_vuln[pr_vuln['repository'].isin(missing_repos)])

54

In [10]:
pr_vuln.head()

,repository,number,url,date,state,package,from,to,vulnerabilities,severities,maximal_severity
0,b2wads/grimorio-ui,74,https://github.com/b2wads/grimorio-ui/pull/74,2019-10-10T19:21:58Z,CLOSED,webpack-bundle-analyzer,3.0.3,3.3.2,['GHSA-pgr8-jg6h-8gw6'],['MODERATE'],MODERATE
1,b2wads/grimorio-ui,107,https://github.com/b2wads/grimorio-ui/pull/107,2020-01-06T16:47:21Z,MERGED,handlebars,4.1.2,4.5.3,['GHSA-w457-6q6x-cgp9'],['HIGH'],HIGH
2,idena-network/idena-desktop,1,https://github.com/idena-network/idena-desktop...,2019-06-29T11:57:09Z,MERGED,js-yaml,3.12.1,3.13.1,"['GHSA-2pr6-76vf-7546', 'GHSA-8j8c-7jfh-h6hx']","['MODERATE', 'HIGH']",HIGH
3,idena-network/idena-desktop,3,https://github.com/idena-network/idena-desktop...,2019-07-15T13:01:58Z,MERGED,lodash,4.17.11,4.17.14,['GHSA-jf85-cpcp-j695'],['HIGH'],HIGH
4,idena-network/idena-desktop,35,https://github.com/idena-network/idena-desktop...,2019-09-11T10:02:08Z,CLOSED,mixin-deep,1.3.1,1.3.2,['GHSA-fhjf-83wg-r2j9'],['HIGH'],HIGH


In [8]:
pr_vuln[pr_vuln['repository'].isin(missing_repos)]

,repository,number,url,date,state,package,from,to,vulnerabilities,severities,maximal_severity
284,strapi/foodadvisor,19,https://github.com/strapi/foodadvisor/pull/19,2019-07-11T14:23:37Z,CLOSED,lodash.template,4.4.0,4.5.0,['GHSA-jf85-cpcp-j695'],['HIGH'],HIGH
285,strapi/foodadvisor,20,https://github.com/strapi/foodadvisor/pull/20,2019-07-13T13:58:04Z,CLOSED,lodash,4.17.11,4.17.13,['GHSA-jf85-cpcp-j695'],['HIGH'],HIGH
286,strapi/foodadvisor,28,https://github.com/strapi/foodadvisor/pull/28,2019-10-21T18:13:06Z,CLOSED,knex,0.16.3,0.19.5,['GHSA-58v4-qwx5-7f59'],['HIGH'],HIGH
287,strapi/foodadvisor,29,https://github.com/strapi/foodadvisor/pull/29,2019-10-25T18:13:20Z,CLOSED,mongoose,5.7.3,5.7.7,['GHSA-8687-vv9j-hgph'],['MODERATE'],MODERATE
363,refugies-info/karfur,3,https://github.com/refugies-info/karfur/pull/3,2019-10-29T16:09:37Z,CLOSED,mongoose,5.4.13,5.7.5,['GHSA-8687-vv9j-hgph'],['MODERATE'],MODERATE
364,refugies-info/karfur,4,https://github.com/refugies-info/karfur/pull/4,2019-12-29T04:54:16Z,CLOSED,handlebars,4.0.12,4.5.3,"['GHSA-w457-6q6x-cgp9', 'GHSA-q42p-pg8m-cqh6']","['HIGH', 'CRITICAL']",CRITICAL
485,imvetri/ui-editor,254,https://github.com/imvetri/ui-editor/pull/254,2020-03-10T14:41:02Z,MERGED,serialize-javascript,1.7.0,2.1.1,['GHSA-h9rv-jmmf-4pgx'],['MODERATE'],MODERATE
486,imvetri/ui-editor,260,https://github.com/imvetri/ui-editor/pull/260,2020-03-14T15:28:52Z,MERGED,acorn,6.2.1,6.4.1,['GHSA-6chw-6frg-f759'],['MODERATE'],MODERATE
554,sharetribe/flex-docs,242,https://github.com/sharetribe/flex-docs/pull/242,2020-03-14T02:54:42Z,CLOSED,acorn,6.3.0,6.4.1,['GHSA-6chw-6frg-f759'],['MODERATE'],MODERATE
689,hull/hull-connectors,290,https://github.com/hull/hull-connectors/pull/290,2019-11-01T22:06:19Z,CLOSED,csv-parse,2.5.0,4.6.5,['GHSA-582f-p4pg-xc74'],['HIGH'],HIGH


In [15]:
import csv
import sys
import pandas as pd
from ast import literal_eval
import subprocess
import json
#import execjs
import multiprocessing as mp
from tqdm import tqdm
sys.setrecursionlimit(10000)

sys.path.append('..')
from finder import *  # noqa: E402
from vulnerability_fixes.graph import NetworkGraph  # noqa: E402
from vulnerability_fixes.identify import identify_applicable_vulnerabilities  # noqa: E402
#from vulnerability_fixes.yarn_lock import yarn_lock_vulnerable  # noqa: E402
from vulnerability_fixes.package_json import package_json_vulnerable, package_json_workspaces  # noqa: E402
#from vulnerability_fixes.package_lock_json import package_lock_json_vulnerable  # noqa: E402

In [17]:
class Fixer:
    def __init__(self, graph):
        self.visited = dict.fromkeys(graph.commits.keys(), False)
        self.fixed = False
        self.fixed_commits = []

    def visit(self, sha):
        self.visited[sha] = True

    def is_visited(self, sha):
        return self.visited[sha]

    def add_fixed_commit(self, sha):
        self.fixed = True
        self.fixed_commits.append(sha)


def load_advisory_database():
    database = pd.read_csv(PATH_GITHUB_ADVISORIES['modified'],
                           index_col=False,
                           dtype={
                             "package": str,
                             "ecosystem": str,
                             "severity": str,
                             "cve": str,
                             "ghsa": str,
                             "summary": str,
                             "publishedAt": str,
                             "updatedAt": str,
                             "firstPatchedVersion": str,
                             "vulnerableVersionRange": str
                           }).assign(
        publishedAt=lambda d: pd.to_datetime(d['publishedAt'], infer_datetime_format=True),
        updatedAt=lambda d: pd.to_datetime(d['updatedAt'], infer_datetime_format=True)
    )
    npm_mask = database['ecosystem'] == 'NPM'
    return database[npm_mask]


def load_update_commits():
    upd_commits = pd.read_csv(PATH_REPOSITORIES_DATA['security_updates_commits'], index_col=False)
    upd_commits['files'] = upd_commits['files'].apply(literal_eval)
    return upd_commits


def load_update_vulnerabilities():
    upd_vulns = pd.read_csv(PATH_REPOSITORIES_DATA['pr_vulnerabilities'], index_col=False)
    upd_vulns['vulnerabilities'] = upd_vulns['vulnerabilities'].apply(literal_eval)
    return upd_vulns


def associate_commits_with_vulns():
    upd_commits = load_update_commits()
    upd_vulns = load_update_vulnerabilities()
    merged = upd_commits.merge(upd_vulns, how='inner', on=['repository', 'number', 'url', 'state'])
    return merged


def filter_updates(df, mode):
    mask_merged = df['state'] == 'MERGED'
    mask_known = df['maximal_severity'].isnull()
    if mode:
        return df[mask_merged & ~mask_known].copy()
    else:
        return df[~mask_known].copy()


def is_vulnerable(database, commit_path, dirs):
    affected = False
    vulnerabilities = []
    for directory in dirs:
        is_pj_affected = False
        is_yl_affected = False
        is_plj_affected = False
        is_nsj_affected = False
        is_present = False
        constraint = []
        path_file = os.path.join(commit_path, directory, 'package.json')
        try:
            is_present, constraint, is_pj_affected, pj_vulnerabilities_ghsa, pj_vulnerabilities_ranges, \
            workspaces = package_json_vulnerable(path_file, affected_package, database, js)
            vulnerabilities.extend(pj_vulnerabilities_ghsa)
            if workspaces:
                ad_constraints = package_json_workspaces(os.path.join(commit_path, directory), workspaces,
                                                         affected_package)
                if ad_constraints:
                    is_present = True
                    constraint.extend(ad_constraints)
                    constraint = list(set(constraint))
        except (FileNotFoundError, json.JSONDecodeError):
            pass
        try:
            is_yl_affected, yl_vulnerabilities_ghsa, _ = yarn_lock_vulnerable(
                os.path.join(commit_path, directory, 'yarn.lock'), affected_package, is_present,
                constraint, database, js)
            vulnerabilities.extend(yl_vulnerabilities_ghsa)
        except (FileNotFoundError, json.JSONDecodeError, execjs._exceptions.ProgramError):
            pass
        try:
            is_plj_affected, plj_vulnerabilities_ghsa, _ = \
                package_lock_json_vulnerable(
                    os.path.join(commit_path, directory, 'package-lock.json'), affected_package,
                    is_present, database, js)
            vulnerabilities.extend(plj_vulnerabilities_ghsa)
        except (FileNotFoundError, json.JSONDecodeError):
            pass
        try:
            is_nsj_affected, nsj_vulnerabilities_ghsa, _ = \
                package_lock_json_vulnerable(
                    os.path.join(commit_path, directory, 'npm-shrinkwrap.json'), affected_package,
                    is_present, database, js)
            vulnerabilities.extend(nsj_vulnerabilities_ghsa)
        except (FileNotFoundError, json.JSONDecodeError):
            pass
        affected = affected | is_yl_affected | is_pj_affected | is_plj_affected | is_nsj_affected
    return set(vulnerabilities), affected


def find_fix(database, vulnerabilities, repository, commit, dirs, url, number, state, package, writer):
    if len(vulnerabilities) < 2:
        fixer = Fixer(dict_graphs[repository])
        fixer.visit(commit)
        vulnerability = list(vulnerabilities)[0]
        # last_commit_path = os.path.join(DIR_FREEZE, repository, dict_graphs[repository].def_branch_commits[-1])
        # _, last_commit_affected = is_vulnerable(database, last_commit_path, dirs)
        # if last_commit_affected:
        #     writer.writerow([repository, number, url, package, vulnerability, state, fixer.fixed, fixer.fixed_commits])
        # else:
        try:
            children = dict_graphs[repository].get_children(commit)
        except KeyError:  # no children
            children = []
            # writer.writerow([repository, number, url, package, vulnerability, False, []])
        try:
            for child in children:
                find_fix_recursive(database[database['ghsa'] == vulnerability], repository, child, dirs, fixer,
                                   dict_graphs[repository])
        except (RuntimeError, RecursionError) as err:
            print('Could not process - {} due to {}'.format(row['url'], err))
        writer.writerow([repository, number, url, package, vulnerability, state, fixer.fixed, fixer.fixed_commits])
    else:
        q = mp.Queue()
        processes = [mp.Process(target=find_fix_process, args=(database[database['ghsa'] == vulnerability],
                                                               vulnerability, repository, commit, dirs, url,
                                                               number, state, package, q)) for vulnerability in
                     vulnerabilities]
        for p in processes:
            p.start()
        for p in processes:
            p.join()
        for p in processes:
            writer.writerow(q.get())


def find_fix_process(database, vulnerability, repository, commit, dirs, url, number, state, package, q):
    # print(vulnerability)
    fixer = Fixer(dict_graphs[repository])
    fixer.visit(commit)
    # last_commit_path = os.path.join(DIR_FREEZE, repository, dict_graphs[repository].def_branch_commits[-1])
    # _, last_commit_affected = is_vulnerable(database, last_commit_path, dirs)
    # if last_commit_affected:
    #     # print([repository, number, url, package, vulnerability, state, fixer.fixed, fixer.fixed_commits])
    #     q.put([repository, number, url, package, vulnerability, state, fixer.fixed, fixer.fixed_commits])
    # else:
    try:
        children = dict_graphs[repository].get_children(commit)
    except KeyError:  # no children
        children = []
        # writer.writerow([repository, number, url, package, vulnerability, False, []])
    try:
        for child in children:
            find_fix_recursive(database[database['ghsa'] == vulnerability], repository, child, dirs, fixer,
                               dict_graphs[repository])
        # q.put([repository, number, url, package, vulnerability, state, fixer.fixed, fixer.fixed_commits])
    except (RuntimeError, RecursionError) as err:
        print('Could not process - {} due to {}'.format(row['url'], err))
    q.put([repository, number, url, package, vulnerability, state, fixer.fixed, fixer.fixed_commits])
    # print([repository, number, url, package, vulnerability, state, fixer.fixed, fixer.fixed_commits])
        # fixed_commits = []
        # for fixed_commit in fixer.fixed_commits:
        #     print(graph.get_famility_tree(fixed_commit))
        #     # if (not (graph.belongs_to_family(fixed_commit) in fixer.fixed_commits)) or \
        #     #         (graph.is_def_branch(fixed_commit)):
        #     #     fixed_commits.append(fixed_commit)
        # print(repository)
        # print(database[database['ghsa'] == vulnerability])
        # print(fixer.fixed)
        # print(fixer.fixed_commits)


def find_fix_recursive(database, repository, commit, dirs, fixer, graph):
    affected = True
    if not fixer.is_visited(commit):
        fixer.visit(commit)
        commit_path = os.path.join(DIR_FREEZE, repository, commit)
        if os.path.exists(commit_path):
            vulnerabilities, affected = is_vulnerable(database, commit_path, dirs)
        if affected:
            try:
                for child in graph.get_children(commit):
                    find_fix_recursive(database, repository, child, dirs, fixer, graph)
            except KeyError:  # no children
                pass
        else:
            fixer.add_fixed_commit(commit)

In [19]:
df_advisories = load_advisory_database()
df_updates = associate_commits_with_vulns()
df_updates = filter_updates(df_updates, False)
df_updates = df_updates.tail(len(df_updates.index) - 1101 - 915)
# df_updates = df_updates[df_updates['repository'] == 'ovh/manager']
# df_updates = df_updates.head(5)
# df_updates = df_updates.tail(len(df_updates.index)-612 - 487).head(1)
repositories = set(df_updates['repository'].to_list())
dict_graphs = dict()
print('initializing network graphs...')
for repo in tqdm(repositories):
    dict_graphs[repo] = NetworkGraph(repo)
print('finished')
print('performing the search...')

  1%|          | 4/539 [00:00<00:14, 35.92it/s]

initializing network graphs...


100%|██████████| 539/539 [00:30<00:00, 17.68it/s]

finished
performing the search...


In [21]:
df_updates[df_updates['repository'].isin(missing_repos)]

,repository,number,url,parent_oid,parent_time,commit_oid,commit_time,final_oid,final_time,rebased,...,created_at,closed_at,files,date,package,from,to,vulnerabilities,severities,maximal_severity
3824,evt-project/evt-viewer,43,https://github.com/evt-project/evt-viewer/pull/43,307d0328f4fe6b52b59c784380855b08dac809ca,2018-06-23T11:20:39Z,37c4a4fbade172ad05a27b855ac11f046219ccc9,2019-11-02T07:26:29Z,5990d8723f1259b18a44b688d425019a18db9f47,2020-07-01T12:01:43Z,True,...,2019-11-02T07:26:29Z,2020-07-01T12:16:22Z,"[package-lock.json, package.json]",2019-11-02T07:26:29Z,bower,1.8.2,1.8.8,[GHSA-p6mr-pxg4-68hx],['HIGH'],HIGH


In [26]:
dfff[dfff['repository'].isin(missing_repos)]

,repository,number,url,parent_oid,parent_time,commit_oid,commit_time,final_oid,final_time,rebased,...,created_at,closed_at,files,date,package,from,to,vulnerabilities,severities,maximal_severity
283,strapi/foodadvisor,19,https://github.com/strapi/foodadvisor/pull/19,2116dfb9973de4319d019882065cf1ec4fa212ad,2019-07-01T06:29:56Z,386d8e5d5d92e0a74caeacf05264726c88be3285,2019-07-11T14:23:37Z,386d8e5d5d92e0a74caeacf05264726c88be3285,2019-07-11T14:23:37Z,False,...,2019-07-11T14:23:37Z,2019-07-30T12:09:28Z,[api/yarn.lock],2019-07-11T14:23:37Z,lodash.template,4.4.0,4.5.0,[GHSA-jf85-cpcp-j695],['HIGH'],HIGH
284,strapi/foodadvisor,20,https://github.com/strapi/foodadvisor/pull/20,2116dfb9973de4319d019882065cf1ec4fa212ad,2019-07-01T06:29:56Z,8f31a81480137b2d206611625d931b79870df4f0,2019-07-13T13:58:04Z,8f31a81480137b2d206611625d931b79870df4f0,2019-07-13T13:58:04Z,False,...,2019-07-13T13:58:04Z,2019-07-30T12:09:29Z,"[api/package.json, api/yarn.lock]",2019-07-13T13:58:04Z,lodash,4.17.11,4.17.13,[GHSA-jf85-cpcp-j695],['HIGH'],HIGH
285,strapi/foodadvisor,28,https://github.com/strapi/foodadvisor/pull/28,d087ce2817abac613a2b2d66a09ae9da64bfd5f7,2019-10-05T09:43:33Z,6eccaaedbdd2d3eb9f4383f12f04bd88af07927c,2019-10-21T18:13:06Z,6eccaaedbdd2d3eb9f4383f12f04bd88af07927c,2019-10-21T18:13:06Z,False,...,2019-10-21T18:13:06Z,2019-10-26T18:35:31Z,"[api/package.json, api/yarn.lock]",2019-10-21T18:13:06Z,knex,0.16.3,0.19.5,[GHSA-58v4-qwx5-7f59],['HIGH'],HIGH
286,strapi/foodadvisor,29,https://github.com/strapi/foodadvisor/pull/29,d087ce2817abac613a2b2d66a09ae9da64bfd5f7,2019-10-05T09:43:33Z,7581f9fd4d537da1c5bf6edce61254222cb79580,2019-10-25T18:13:19Z,7581f9fd4d537da1c5bf6edce61254222cb79580,2019-10-25T18:13:19Z,False,...,2019-10-25T18:13:20Z,2019-10-26T18:35:31Z,[api/yarn.lock],2019-10-25T18:13:20Z,mongoose,5.7.3,5.7.7,[GHSA-8687-vv9j-hgph],['MODERATE'],MODERATE
362,refugies-info/karfur,3,https://github.com/refugies-info/karfur/pull/3,44b072f311a45da6707541ea9e0687e964d17fd1,2019-10-14T07:57:58Z,4ab5263d46e6b57e47d9c23e0555d7e20c2e78e7,2019-10-29T16:09:37Z,4ab5263d46e6b57e47d9c23e0555d7e20c2e78e7,2019-10-29T16:09:37Z,False,...,2019-10-29T16:09:37Z,2020-06-09T08:08:55Z,"[package-lock.json, package.json]",2019-10-29T16:09:37Z,mongoose,5.4.13,5.7.5,[GHSA-8687-vv9j-hgph],['MODERATE'],MODERATE
363,refugies-info/karfur,4,https://github.com/refugies-info/karfur/pull/4,c3653e2c95639b6028213a956725fe97e0872da4,2019-12-24T20:34:05Z,8afb10198c3219735d63c4782b7c086a4d6baae2,2019-12-29T04:54:16Z,8afb10198c3219735d63c4782b7c086a4d6baae2,2019-12-29T04:54:16Z,False,...,2019-12-29T04:54:16Z,2020-05-19T12:57:26Z,[client/yarn.lock],2019-12-29T04:54:16Z,handlebars,4.0.12,4.5.3,"[GHSA-w457-6q6x-cgp9, GHSA-q42p-pg8m-cqh6]","['HIGH', 'CRITICAL']",CRITICAL
484,imvetri/ui-editor,254,https://github.com/imvetri/ui-editor/pull/254,1dd959195e8ed1a41079b5aa85032d7e392925b5,2020-03-10T14:40:26Z,e0260e4e060d76c587cf8cc3a174b9a72e706761,2020-03-10T14:41:02Z,e0260e4e060d76c587cf8cc3a174b9a72e706761,2020-03-10T14:41:02Z,False,...,2020-03-10T14:41:02Z,2020-03-12T04:11:44Z,[package-lock.json],2020-03-10T14:41:02Z,serialize-javascript,1.7.0,2.1.1,[GHSA-h9rv-jmmf-4pgx],['MODERATE'],MODERATE
485,imvetri/ui-editor,260,https://github.com/imvetri/ui-editor/pull/260,f12fe0031b7bcd609804712320d1de70d41be782,2020-03-13T13:29:45Z,e6684f036ac8b20c8ad781766ec991ad29cac37a,2020-03-14T15:28:52Z,e6684f036ac8b20c8ad781766ec991ad29cac37a,2020-03-14T15:28:52Z,False,...,2020-03-14T15:28:52Z,2020-03-21T04:12:31Z,[package-lock.json],2020-03-14T15:28:52Z,acorn,6.2.1,6.4.1,[GHSA-6chw-6frg-f759],['MODERATE'],MODERATE
1070,18xx-maker/18xx-maker,227,https://github.com/18xx-maker/18xx-maker/pull/227,c8b9f24d131fc3d4bc4e7acdd4d8cafa5b4bd3e6,2020-03-13T15:43:13Z,732d18ec73177e39b29a024bd4435bad1e77c64e,2020-03-13T23:38:47Z,732d18ec73177e39b29a024bd4435bad1e77c64e,2020-03-13T23:38:47Z,False,...,2020-03-13T23:38:47Z,2020-03-13T23:40:09Z,[yarn.lock],2020-03-13T23:38:47Z,acorn,5.7.3,5.7.4,[GHSA-6chw-6frg-f759],['MODERATE'],M

In [27]:
df_advisories = load_advisory_database()
df_updates = associate_commits_with_vulns()
print(len(df_updates[df_updates['repository'].isin(missing_repos)]))
print(df_updates[df_updates['repository'].isin(missing_repos)])
print('========================')
df_updates = filter_updates(df_updates, False)
print(len(df_updates[df_updates['repository'].isin(missing_repos)]))
print(df_updates[df_updates['repository'].isin(missing_repos)])
print('========================')
df_updates = df_updates.tail(len(df_updates.index) - 1101 - 915)
print(len(df_updates[df_updates['repository'].isin(missing_repos)]))
print(df_updates[df_updates['repository'].isin(missing_repos)])
print('========================')
# df_updates = df_updates[df_updates['repository'] == 'ovh/manager']
# df_updates = df_updates.head(5)
# df_updates = df_updates.tail(len(df_updates.index)-612 - 487).head(1)
repositories = set(df_updates['repository'].to_list())
dict_graphs = dict()
print('initializing network graphs...')
for repo in tqdm(repositories):
    dict_graphs[repo] = NetworkGraph(repo)
print('finished')
print('performing the search...')

  1%|          | 5/539 [00:00<00:11, 46.01it/s]

29
                   repository  number  \
283        strapi/foodadvisor      19   
284        strapi/foodadvisor      20   
285        strapi/foodadvisor      28   
286        strapi/foodadvisor      29   
362      refugies-info/karfur       3   
363      refugies-info/karfur       4   
484         imvetri/ui-editor     254   
485         imvetri/ui-editor     260   
1070    18xx-maker/18xx-maker     227   
1120  alleyinteractive/irving     121   
1121  alleyinteractive/irving     159   
1131        WeBankFinTech/wxa      31   
1132        WeBankFinTech/wxa      32   
1133        WeBankFinTech/wxa      33   
1134        WeBankFinTech/wxa      35   
1135        WeBankFinTech/wxa      36   
1136        WeBankFinTech/wxa      37   
1137        WeBankFinTech/wxa      38   
1138        WeBankFinTech/wxa      39   
1139        WeBankFinTech/wxa      46   
1140        WeBankFinTech/wxa      47   
1141        WeBankFinTech/wxa      48   
1797           mistval/kotoba     117   
1798         

100%|██████████| 539/539 [00:28<00:00, 19.11it/s]

finished
performing the search...
